##  자동 기상 스크래핑

#### ※ 웹 크롤링 및 스크래핑, csv 저장이 동시에 이뤄지기 때문에 작업 중지를 방지하기 위해 단독 실행을 추천합니다.

### Chromedriver

In [ ]:
"""
크롬 드라이버는 사용자의 크롬 버전에 맞게 드라이버를 다운로드 받아 스크래핑 코드와 같은 폴더에 넣어주시면 됩니다. 

크롬 브라우저의 버전은 우측 상단 더보기란 -> 도움말 -> Chrome 정보 를 선택하면 확인할 수 있습니다.

https://chromedriver.chromium.org/downloads 에 가시면 버전에 맞는 크롬 드라이버를 찾아서 다운로드할 수 있습니다.
"""

### Install Selenium

In [1]:
!pip3 install selenium

### 화면 가로 확장

In [14]:
# 화면 가로 확장 코드 (기본 width 50%)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:70% !important; }</style>"))

### 데이터 저장을 위한 폴더 생성

In [6]:
import os
try:
    os.mkdir('./Weather_Data_CSV')
except FileExistsError:
    print('해당 폴더가 이미 존재합니다.\n폴더를 확인해주세요.')

### 정상 시각 일치를 위한 시각과 48개의 순번 Dict

In [3]:
# 11:20 PM부터 데이터가 존재하는 날을 위한 시간 순차 번호
clk_11_48 = {'11:50 PM':1, '12:20 AM':2, '12:50 AM':3, '1:20 AM':4, '1:50 AM':5, '2:20 AM':6,
         '2:50 AM':7, '3:20 AM':8, '3:50 AM':9, '4:20 AM':10, '4:50 AM':11, '5:20 AM':12,
         '5:50 AM':13, '6:20 AM':14, '6:50 AM':15, '7:20 AM':16, '7:50 AM':17, '8:20 AM':18,
         '8:50 AM':19, '9:20 AM':20, '9:50 AM':21, '10:20 AM':22, '10:50 AM':23, '11:20 AM':24,
         '11:50 AM':25, '12:20 PM':26, '12:50 PM':27, '1:20 PM':28, '1:50 PM':29, '2:20 PM':30,
         '2:50 PM':31, '3:20 PM':32, '3:50 PM':33, '4:20 PM':34, '4:50 PM':35, '5:20 PM':36,
         '5:50 PM':37, '6:20 PM':38, '6:50 PM':39, '7:20 PM':40, '7:50 PM':41, '8:20 PM':42,
         '8:50 PM':43, '9:20 PM':44, '9:50 PM':45, '10:20 PM':46, '10:50 PM':47, '11:20 PM':48}

# reversed
re_clk_11_48 = dict(map(reversed, clk_11_48.items()))
print(re_clk_11_48)
print()

{1: '11:50 PM', 2: '12:20 AM', 3: '12:50 AM', 4: '1:20 AM', 5: '1:50 AM', 6: '2:20 AM', 7: '2:50 AM', 8: '3:20 AM', 9: '3:50 AM', 10: '4:20 AM', 11: '4:50 AM', 12: '5:20 AM', 13: '5:50 AM', 14: '6:20 AM', 15: '6:50 AM', 16: '7:20 AM', 17: '7:50 AM', 18: '8:20 AM', 19: '8:50 AM', 20: '9:20 AM', 21: '9:50 AM', 22: '10:20 AM', 23: '10:50 AM', 24: '11:20 AM', 25: '11:50 AM', 26: '12:20 PM', 27: '12:50 PM', 28: '1:20 PM', 29: '1:50 PM', 30: '2:20 PM', 31: '2:50 PM', 32: '3:20 PM', 33: '3:50 PM', 34: '4:20 PM', 35: '4:50 PM', 36: '5:20 PM', 37: '5:50 PM', 38: '6:20 PM', 39: '6:50 PM', 40: '7:20 PM', 41: '7:50 PM', 42: '8:20 PM', 43: '8:50 PM', 44: '9:20 PM', 45: '9:50 PM', 46: '10:20 PM', 47: '10:50 PM', 48: '11:20 PM'}



### 월 마다 다른 일수 반영, 48분할 하루의 온도를 스크래핑, 결측값 정상 입력 함수

In [33]:
def Monthly_Temp(YYYY, MM):
    driver.implicitly_wait(10)
    YYYY = y
    MM = m

    # 각 달마다 일수가 다르고 하루 단위로 동작되는 코드는 중복되는 내용이 많기 때문에 간소화를 위해 월 Dict 이용
    month = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
    d=0
    
    global temp_t_x
    global temp_c_x
    global temp_t
    global temp_c
    
    
    
    while(d<month[m]):
        d=d+1
        print('====================< %d일 >=========================' %d)
        print()
        driver.get('https://www.wunderground.com/history/daily/gb/london/EGLC/date/'+str(y)+'-'+str(m)+'-'+str(d))
        driver.implicitly_wait(10)
        time.sleep(3)
                
        h = 1 # 여기서는 하루를 30분 단위로 쪼갠 온도 개수, 데이터의 개수를 h로 설정해서 반복시켰는데, 48이 안되는 경우도 있음.
        h2 = 1
        
        clk = driver.find_element(By.XPATH,'//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table/tbody/tr['+str(h2)+']/td[1]/span')
        clk_str = clk.text
        clk_str.split()

        
        # 11:50 PM 이전 시각부터 시작하는 경우
        # 42는 적당히 오차 범위일거라 생각하는 숫자로 임의 지정한 것이지만 42 ~ 48 까지는 정상 시각 시작전을 뜻함.
        if(int(clk_11_48[clk_str])>=42):
            
            print('<11:50 PM 이전 시각부터 데이터 시작>')
            x = 0
            while(h2 < 49):
                clk = driver.find_element(By.XPATH,'//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table/tbody/tr['+str(h2)+']/td[1]/span')
                clk_str = clk.text
                clk_str.split()

                if(clk_str == '11:50 PM'):
                    x = 1
                    print('x == 1\n')
                
                # 11:50 PM에 걸렸다면 해당 경우로 진행되고 안 걸렸다면 그다음으로 넘어가기 때문에 여기서 x == 2로 설정됨. 이후도 마찬가지.
                if(clk_str == '12:20 AM'):
                    x = 2
                    print('x == 2\n')
                
                # 중복이 많아서 clk_11_48[clk_str]로 처리하려 했으나 이전 시각대에서 47같은 숫자가 나와서 하나씩 설정함.   
                if(clk_str == '12:50 AM'):
                    x = 3
                    print('x == 3\n')
                    
                if(clk_str == '1:20 AM'):
                    x = 4
                    print('x == 4\n')
                    
                if(clk_str == '1:50 AM'):
                    x = 5
                    print('x == 5\n')
                    
                if(clk_str == '2:20 AM'):
                    x = 6
                    print('x == 6\n')
                    
                if(clk_str == '2:50 AM'):
                    x = 7
                    print('x == 7\n')
                    
                if(clk_str == '3:20 AM'):
                    x = 8
                    print('x == 8\n')
                    
                if(clk_str == '3:50 AM'):
                    x = 9
                    print('x == 9\n')
                    
                if(clk_str == '4:20 AM'):
                    x = 10
                    print('x == 10\n')

                
                if(x == 1):
                    h3 = 1
                    while (h3 < 49):
                        try:
                            clk2 = driver.find_element(By.XPATH,'//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table/tbody/tr['+str(h2)+']/td[1]/span')
                            clk_str2 = clk2.text
                            clk_str2.split()
                            
                            # 5:01 과 같은 시각을 걸러내기 위해 dict 안에 있는지 검사
                            if(clk_str2 in clk_11_48):
                                
                                # 이 if문 진입전에는 h3 = 1로 고정된 상태.
                                if(h3==clk_11_48[clk_str2]):
                                    date_list = [int(y), int(m), int(d)] # 연,월,일 정보로 구성된 날짜 리스트 생성
                                    date_list.append(clk_str2) # 날짜 리스트에 해당 시각 원소 추가
                                    date_list.append(int(h3)) # 해당 시각이 그날 48개로 분할된 시간 중 몇번째인지 원소 추가
                                    temp = driver.find_element(By.XPATH,'//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table/tbody/tr['+str(h2)+']/td[2]')
                                    temp_str = temp.text # 스크래핑한 xpath의 온도를 str로 변경
                                    temp_str.split
                                    temp_t=temp_str[:2] # temp_str 에서 온도 숫자만 추출
                                    temp_c=temp_str[2:]
                                    temp_t_x = temp_t
                                    temp_c_x = temp_c
                                    temp_list = date_list 
                                    temp_list.append(temp_t_x) # 날짜+시각 리스트에 온도 원소 추가
                                    temp_list.append(temp_c_x) # 날짜+시각 리스트에 섭씨 기호 원소 추가
                                    time.sleep(0.02)
                                    print(temp_list) # 확인용 print
                                    # 시간 루프 종료 지점

                                    # csv 추가 및 저장
                                    with open("./Weather_Data_CSV/"+str(y)+'_'+str(m)+'_'+"London_Temperature.csv", 'a', newline='') as file:
                                        writer = csv.writer(file)
                                        writer.writerow(temp_list)
                                    print()
                                    h2 = h2+1
                                    h3 = h3+1

                                else:
                                    date_list = [int(y), int(m), int(d)]
                                    print('결손 데이터 칸의 정상 시각 >>',re_clk_11_48[h])
                                    print('사이트에서 스크래핑한 시각 >>', clk_str2)
                                    date_list.append(re_clk_11_48[h])
                                    date_list.append(int(h3))
                                    na_list = date_list
                                    na_list.append(temp_t) # 직전 시각의 온도를 그대로 입력
                                    na_list.append(temp_c)
                                    print(na_list)
                                    h3 = h3+1

                                    with open("./Weather_Data_CSV/"+str(y)+'_'+str(m)+'_'+"London_Temperature.csv", 'a', newline='') as file:
                                        writer = csv.writer(file)
                                        writer.writerow(na_list)
                                    print() 

                                h = h+1 
                                
                            else:
                                for E in range(1):
                                    h2 = h2+1

                        except NoSuchElementException:
                            if(h<49):
                                # 맨 마지막에 데이터가 없어 조기 종료 되버리는 경우
                                date_list = [int(y), int(m), int(d)]
                                print('결손 데이터 칸의 정상 시각 >>',re_clk_11_48[h]) 
                                print('사이트에서 스크래핑한 시각 >>', clk_str)
                                date_list.append(re_clk_11_48[h])
                                date_list.append(int(h))
                                na_list = date_list
                                na_list.append(temp_t)
                                na_list.append(temp_c)
                                print(na_list)

                                with open("./Weather_Data_CSV/"+str(y)+'_'+str(m)+'_'+"London_Temperature.csv", 'a', newline='') as file:
                                    writer = csv.writer(file)
                                    writer.writerow(na_list)
                                    print() 
                                h = h+1
                            else:
                                break
                    break
                
                
                # 처음에는 11:50 PM 이전 시작 11:50 PM 결손. 그리고 12:20 AM 부터 시작되는 경우만 생각했는데 11:50 PM 부터 쭉 데이터가 없는 경우를 생각해서 만듬.
                if(x >= 2):
                    z = 1
                    while(z < x):
                        date_list = [int(y), int(m), int(d)]
                        print('결손 데이터 칸의 정상 시각 >>',re_clk_11_48[h])
                        print('11:50 PM 이전 시각에 데이터 시작했지만 11:50 PM 데이터 결손')
                        date_list.append(re_clk_11_48[h])
                        date_list.append(z)
                        na_list = date_list
                        na_list.append(temp_t_x) # 직전 시각의 온도를 그대로 입력
                        na_list.append(temp_c_x)
                        print(na_list)
                        
                        print()
                        with open("./Weather_Data_CSV/"+str(y)+'_'+str(m)+'_'+"London_Temperature.csv", 'a', newline='') as file:
                            writer = csv.writer(file)
                            writer.writerow(na_list)
                        z = z+1
                        h = h+1
                        
                    h3 = 1 + (x-1)
                    
                    while (h3 < 49):
                        try:
                            clk2 = driver.find_element(By.XPATH,'//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table/tbody/tr['+str(h2)+']/td[1]/span')
                            clk_str2 = clk2.text
                            clk_str2.split()
                            
                            if(clk_str2 in clk_11_48):
                                # 이 if문 진입전에는 h3 = 1로 고정된 상태.
                                if(h3==clk_11_48[clk_str2]):
                                    date_list = [int(y), int(m), int(d)] # 연,월,일 정보로 구성된 날짜 리스트 생성
                                    date_list.append(clk_str2) # 날짜 리스트에 해당 시각 원소 추가
                                    date_list.append(int(h3)) # 해당 시각이 그날 48개로 분할된 시간 중 몇번째인지 원소 추가
                                    temp = driver.find_element(By.XPATH,'//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table/tbody/tr['+str(h2)+']/td[2]')
                                    temp_str = temp.text # 스크래핑한 xpath의 온도를 str로 변경
                                    temp_t=temp_str[:2] # temp_str 에서 온도 숫자만 추출
                                    temp_c=temp_str[2:]
                                    temp_t_x = temp_t
                                    temp_c_x = temp_c
                                    temp_list = date_list 
                                    temp_list.append(temp_t_x) # 날짜+시각 리스트에 온도 원소 추가
                                    temp_list.append(temp_c_x) # 날짜+시각 리스트에 섭씨 기호 원소 추가
                                    time.sleep(0.02)
                                    print(temp_list) # 확인용 print
                                    # 시간 루프 종료 지점

                                    # csv 추가 및 저장
                                    with open("./Weather_Data_CSV/"+str(y)+'_'+str(m)+'_'+"London_Temperature.csv", 'a', newline='') as file:
                                        writer = csv.writer(file)
                                        writer.writerow(temp_list)
                                    print()
                                    h2 = h2+1
                                    h3 = h3+1

                                else:
                                    try:
                                        date_list = [int(y), int(m), int(d)]
                                        print('결손 데이터 칸의 정상 시각 >>',re_clk_11_48[h])
                                        # 11:50 이전에 시작했는데 11:50 PM 시각 데이터도 없는 경우는 처리 방법 미해결.
                                        print('사이트에서 스크래핑한 시각 >>', clk_str2)
                                        date_list.append(re_clk_11_48[h])
                                        date_list.append(int(h3))
                                        na_list = date_list
                                        na_list.append(temp_t) # 직전 시각의 온도를 그대로 입력
                                        na_list.append(temp_c)
                                        print(na_list)
                                        h3 = h3+1

                                        with open("./Weather_Data_CSV/"+str(y)+'_'+str(m)+'_'+"London_Temperature.csv", 'a', newline='') as file:
                                            writer = csv.writer(file)
                                            writer.writerow(na_list)
                                        print() 
                                    except KeyError:
                                        break

                                h = h+1
                            
                            else:
                                for E in range(1):
                                    h2 = h2+1

                        except NoSuchElementException:
                            if(h<49):
                                # 맨 마지막에 데이터가 없어 조기 종료 되버리는 경우
                                date_list = [int(y), int(m), int(d)]
                                print('결손 데이터 칸의 정상 시각 >>',re_clk_11_48[h]) 
                                print('사이트에서 스크래핑한 시각 >>', clk_str)
                                date_list.append(re_clk_11_48[h])
                                date_list.append(int(h))
                                na_list = date_list
                                na_list.append(temp_t)
                                na_list.append(temp_c)
                                print(na_list)

                                with open("./Weather_Data_CSV/"+str(y)+'_'+str(m)+'_'+"London_Temperature.csv", 'a', newline='') as file:
                                    writer = csv.writer(file)
                                    writer.writerow(na_list)
                                    print() 
                                h = h+1
                            else:
                                break
                    break
                    
                else:
                    h2=h2+1
                
                
                
                    
        # 11:50 PM 데이터부터 시작하는 경우
        else:
            while (h2 < 49):
                try:    
                    clk = driver.find_element(By.XPATH,'//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table/tbody/tr['+str(h2)+']/td[1]/span')
                    clk_str = clk.text
                    clk_str.split()

                    
                    if(h==clk_11_48[clk_str]):
                        date_list = [int(y), int(m), int(d)] # 연,월,일 정보로 구성된 날짜 리스트 생성
                        date_list.append(clk_str) # 날짜 리스트에 해당 시각 원소 추가
                        date_list.append(int(h)) # 해당 시각이 그날 48개로 분할된 시간 중 몇번째인지 원소 추가
                        temp = driver.find_element(By.XPATH,'//*[@id="inner-content"]/div[2]/div[1]/div[5]/div[1]/div/lib-city-history-observation/div/div[2]/table/tbody/tr['+str(h2)+']/td[2]')
                        temp_str = temp.text # 스크래핑한 xpath의 온도를 str로 변경
                        temp_str.split
                        temp_t=temp_str[:2] # temp_str 에서 온도 숫자만 추출
                        temp_c=temp_str[2:]
                        temp_t_x = temp_t
                        temp_c_x = temp_c
                        temp_list = date_list 
                        temp_list.append(temp_t) # 날짜+시각 리스트에 온도 원소 추가
                        temp_list.append(temp_c) # 날짜+시각 리스트에 섭씨 기호 원소 추가
                        time.sleep(0.02)
                        print(temp_list) # 확인용 print
                        # 시간 루프 종료 지점
                            
                        # csv 추가 및 저장
                        with open("./Weather_Data_CSV/"+str(y)+'_'+str(m)+'_'+"London_Temperature.csv", 'a', newline='') as file:
                            writer = csv.writer(file)
                            writer.writerow(temp_list)
                        
                        print()
                        h2 = h2+1
                    
                    else:
                        try:
                            date_list = [int(y), int(m), int(d)]
                            print('결손 데이터 칸의 정상 시각 >>',re_clk_11_48[h])
                            print('사이트에서 스크래핑한 시각 >>', clk_str)
                            date_list.append(re_clk_11_48[h])
                            date_list.append(int(h))
                            na_list = date_list
                            na_list.append(temp_t_x) # 직전 시각의 온도를 그대로 입력
                            na_list.append(temp_c_x)
                            print(na_list)

                            with open("./Weather_Data_CSV/"+str(y)+'_'+str(m)+'_'+"London_Temperature.csv", 'a', newline='') as file:
                                writer = csv.writer(file)
                                writer.writerow(na_list)
                            print() 
                        except KeyError:
                            break
                    h = h+1
                    
                except NoSuchElementException:
                    if(h<49):
                        # 맨 마지막에 데이터가 없어 조기 종료 되버리는 경우
                        date_list = [int(y), int(m), int(d)]
                        print('결손 데이터 칸의 정상 시각 >>',re_clk_11_48[h])
                        print('사이트에서 스크래핑한 시각 >>', clk_str)
                        date_list.append(re_clk_11_48[h])
                        date_list.append(int(h))
                        na_list = date_list
                        na_list.append(temp_t)
                        na_list.append(temp_c)
                        print(na_list)
                            
                        with open("./Weather_Data_CSV/"+str(y)+'_'+str(m)+'_'+"London_Temperature.csv", 'a', newline='') as file:
                            writer = csv.writer(file)
                            writer.writerow(na_list)
                            print() 
                        h = h+1
                        
                    else:
                        break
                        
        temp_t_x = temp_t                
        print('< Day End >')
        print('\n'*4)  

#### 2013~2016년의 기상 데이터를 수집하는 전체 코드

In [35]:
# 데이터 저장을 위한 csv 
import csv

# Selenium : 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# 에러 예외처리를 위해 해당 예외를 먼저 임포트해줌.
from selenium.common.exceptions import NoSuchElementException

# 정상작동을 위해서 중간 시간 정보 호출 사이에 강제 딜레이를 잠깐 넣어줌.
import time

path = "./chromedriver.exe"
driver = webdriver.Chrome(path)
driver.implicitly_wait(10)

# 기상포털 주소 1차 접속. 데이터가 충분히 표시된 2021년 1월 1일을 시작 화면으로 설정
driver.get('https://www.wunderground.com/history/daily/gb/london/EGLC/date/2021-1-1')
time.sleep(1)

# 온도 표기 선택창 클릭
driver.find_element(By.XPATH, '//*[@id="wuSettings"]/i').click()
time.sleep(1)

# 섭씨 선택
driver.find_element(By.XPATH, '//*[@id="wuSettings-quick"]/div/a[2]').click()
time.sleep(1)

# YYYY offset 2013
y = 2013
while(y<2017):
    print('====================< %d년 >===================================' %y)
    
    # MM offset 1
    m = 1
    while(m<13):
        
        print('====================< %d월 >==============================' %m)
        
        # DD
        Monthly_Temp(y, m)
        
        temp_t_x = temp_t
        temp_c_x = temp_c
        m=m+1              
        print()
    print('\n')
    
    """
    여기서 당해의 1년 csv 데이터 합본 생성 (2013, 2014, 2015, 2016)
    2013_1_London_Temperature 파일 이름에서
    2013_m_London_Temperature 로 해서 m이 1~12 범위 내에서 반복 결합 되도록
    파일이름은 2013_Unit_London_Temperature
    """
    y=y+1

print('스크래핑 종료.')
driver.quit()

C:\Users\ADMINI~1\AppData\Local\Temp/ipykernel_9224/3029470105.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


====================< 2014년 >===================================
====================< 10월 >==============================
====================< 1일 >=========================

<11:50 PM 이전 시각부터 데이터 시작>
x == 1

[2014, 10, 1, '11:50 PM', 1, '16', ' °C']

[2014, 10, 1, '12:20 AM', 2, '16', ' °C']

[2014, 10, 1, '12:50 AM', 3, '16', ' °C']

[2014, 10, 1, '1:20 AM', 4, '16', ' °C']

[2014, 10, 1, '1:50 AM', 5, '16', ' °C']

[2014, 10, 1, '2:20 AM', 6, '17', ' °C']

[2014, 10, 1, '2:50 AM', 7, '17', ' °C']

[2014, 10, 1, '3:20 AM', 8, '17', ' °C']

[2014, 10, 1, '3:50 AM', 9, '17', ' °C']

[2014, 10, 1, '4:20 AM', 10, '17', ' °C']

[2014, 10, 1, '4:50 AM', 11, '17', ' °C']

[2014, 10, 1, '5:20 AM', 12, '18', ' °C']

[2014, 10, 1, '5:50 AM', 13, '18', ' °C']

[2014, 10, 1, '6:20 AM', 14, '18', ' °C']

[2014, 10, 1, '6:50 AM', 15, '18', ' °C']

[2014, 10, 1, '7:20 AM', 16, '18', ' °C']

[2014, 10, 1, '7:50 AM', 17, '18', ' °C']

[2014, 10, 1, '8:20 AM', 18, '17', ' °C']

[2014, 10, 1, '8:50 AM

[2014, 10, 4, '2:20 PM', 30, '13', ' °C']

[2014, 10, 4, '2:50 PM', 31, '13', ' °C']

[2014, 10, 4, '3:20 PM', 32, '13', ' °C']

[2014, 10, 4, '3:50 PM', 33, '13', ' °C']

[2014, 10, 4, '4:20 PM', 34, '13', ' °C']

[2014, 10, 4, '4:50 PM', 35, '13', ' °C']

[2014, 10, 4, '5:20 PM', 36, '13', ' °C']

[2014, 10, 4, '5:50 PM', 37, '12', ' °C']

[2014, 10, 4, '6:20 PM', 38, '12', ' °C']

[2014, 10, 4, '6:50 PM', 39, '12', ' °C']

[2014, 10, 4, '7:20 PM', 40, '12', ' °C']

[2014, 10, 4, '7:50 PM', 41, '11', ' °C']

[2014, 10, 4, '8:20 PM', 42, '11', ' °C']

[2014, 10, 4, '8:50 PM', 43, '11', ' °C']

[2014, 10, 4, '9:20 PM', 44, '12', ' °C']

[2014, 10, 4, '9:50 PM', 45, '11', ' °C']

[2014, 10, 4, '10:20 PM', 46, '11', ' °C']

[2014, 10, 4, '10:50 PM', 47, '11', ' °C']

결손 데이터 칸의 정상 시각 >> 11:20 PM
사이트에서 스크래핑한 시각 >> 11:50 PM
[2014, 10, 4, '11:20 PM', 48, '11', ' °C']

< Day End >





====================< 5일 >=========================

<11:50 PM 이전 시각부터 데이터 시작>
x == 1

[2014, 10, 5, '11:50 


[2014, 10, 8, '5:50 AM', 13, '12', ' °C']

[2014, 10, 8, '6:20 AM', 14, '12', ' °C']

[2014, 10, 8, '6:50 AM', 15, '12', ' °C']

[2014, 10, 8, '7:20 AM', 16, '13', ' °C']

[2014, 10, 8, '7:50 AM', 17, '13', ' °C']

[2014, 10, 8, '8:20 AM', 18, '13', ' °C']

[2014, 10, 8, '8:50 AM', 19, '14', ' °C']

[2014, 10, 8, '9:20 AM', 20, '14', ' °C']

[2014, 10, 8, '9:50 AM', 21, '14', ' °C']

[2014, 10, 8, '10:20 AM', 22, '14', ' °C']

[2014, 10, 8, '10:50 AM', 23, '15', ' °C']

[2014, 10, 8, '11:20 AM', 24, '16', ' °C']

[2014, 10, 8, '11:50 AM', 25, '17', ' °C']

[2014, 10, 8, '12:20 PM', 26, '17', ' °C']

[2014, 10, 8, '12:50 PM', 27, '17', ' °C']

[2014, 10, 8, '1:20 PM', 28, '17', ' °C']

[2014, 10, 8, '1:50 PM', 29, '18', ' °C']

[2014, 10, 8, '2:20 PM', 30, '18', ' °C']

[2014, 10, 8, '2:50 PM', 31, '17', ' °C']

[2014, 10, 8, '3:20 PM', 32, '18', ' °C']

[2014, 10, 8, '3:50 PM', 33, '17', ' °C']

[2014, 10, 8, '4:20 PM', 34, '17', ' °C']

[2014, 10, 8, '4:50 PM', 35, '17', ' °C']

[201


[2014, 10, 11, '8:50 PM', 43, '12', ' °C']

[2014, 10, 11, '9:20 PM', 44, '12', ' °C']

[2014, 10, 11, '9:50 PM', 45, '12', ' °C']

[2014, 10, 11, '10:20 PM', 46, '12', ' °C']

[2014, 10, 11, '10:50 PM', 47, '11', ' °C']

결손 데이터 칸의 정상 시각 >> 11:20 PM
사이트에서 스크래핑한 시각 >> 11:50 PM
[2014, 10, 11, '11:20 PM', 48, '11', ' °C']

< Day End >





====================< 12일 >=========================

<11:50 PM 이전 시각부터 데이터 시작>
x == 1

[2014, 10, 12, '11:50 PM', 1, '11', ' °C']

[2014, 10, 12, '12:20 AM', 2, '11', ' °C']

[2014, 10, 12, '12:50 AM', 3, '10', ' °C']

[2014, 10, 12, '1:20 AM', 4, '11', ' °C']

[2014, 10, 12, '1:50 AM', 5, '10', ' °C']

[2014, 10, 12, '2:20 AM', 6, '10', ' °C']

[2014, 10, 12, '2:50 AM', 7, '10', ' °C']

[2014, 10, 12, '3:20 AM', 8, '10', ' °C']

[2014, 10, 12, '3:50 AM', 9, '10', ' °C']

[2014, 10, 12, '4:20 AM', 10, '9 ', '°C']

[2014, 10, 12, '4:50 AM', 11, '9 ', '°C']

[2014, 10, 12, '5:20 AM', 12, '9 ', '°C']

[2014, 10, 12, '5:50 AM', 13, '8 ', '°C']

[2014, 10,

...


13  °C
[2014, 10, 29, '11:50 AM', 25, '13', ' °C']

13  °C
[2014, 10, 29, '12:20 PM', 26, '13', ' °C']

13  °C
[2014, 10, 29, '12:50 PM', 27, '13', ' °C']

13  °C
[2014, 10, 29, '1:20 PM', 28, '13', ' °C']

13  °C
[2014, 10, 29, '1:50 PM', 29, '13', ' °C']

13  °C
[2014, 10, 29, '2:20 PM', 30, '13', ' °C']

13  °C
[2014, 10, 29, '2:50 PM', 31, '13', ' °C']

13  °C
[2014, 10, 29, '3:20 PM', 32, '13', ' °C']

13  °C
[2014, 10, 29, '3:50 PM', 33, '13', ' °C']

13  °C
[2014, 10, 29, '4:20 PM', 34, '13', ' °C']

13  °C
[2014, 10, 29, '4:50 PM', 35, '13', ' °C']

13  °C
[2014, 10, 29, '5:20 PM', 36, '13', ' °C']

13  °C
[2014, 10, 29, '5:50 PM', 37, '13', ' °C']

13  °C
[2014, 10, 29, '6:20 PM', 38, '13', ' °C']

13  °C
[2014, 10, 29, '6:50 PM', 39, '13', ' °C']

13  °C
[2014, 10, 29, '7:20 PM', 40, '13', ' °C']

13  °C
[2014, 10, 29, '7:50 PM', 41, '13', ' °C']

13  °C
[2014, 10, 29, '8:20 PM', 42, '13', ' °C']

13  °C
[2014, 10, 29, '8:50 PM', 43, '13', ' °C']

13  °C
[2014, 10, 29, '9:20


16  °C
[2014, 11, 1, '4:50 PM', 35, '16', ' °C']

15  °C
[2014, 11, 1, '5:20 PM', 36, '15', ' °C']

15  °C
[2014, 11, 1, '5:50 PM', 37, '15', ' °C']

14  °C
[2014, 11, 1, '6:20 PM', 38, '14', ' °C']

14  °C
[2014, 11, 1, '6:50 PM', 39, '14', ' °C']

14  °C
[2014, 11, 1, '7:20 PM', 40, '14', ' °C']

14  °C
[2014, 11, 1, '7:50 PM', 41, '14', ' °C']

14  °C
[2014, 11, 1, '8:20 PM', 42, '14', ' °C']

14  °C
[2014, 11, 1, '8:50 PM', 43, '14', ' °C']

14  °C
[2014, 11, 1, '9:20 PM', 44, '14', ' °C']

14  °C
[2014, 11, 1, '9:50 PM', 45, '14', ' °C']

14  °C
[2014, 11, 1, '10:20 PM', 46, '14', ' °C']

14  °C
[2014, 11, 1, '10:50 PM', 47, '14', ' °C']

15  °C
[2014, 11, 1, '11:20 PM', 48, '15', ' °C']

< Day End >





====================< 2일 >=========================

결손 데이터 칸의 정상 시각 >> 11:50 PM
사이트에서 스크래핑한 시각 >> 12:20 AM
[2014, 11, 2, '11:50 PM', 1, '15', ' °C']

15  °C
[2014, 11, 2, '12:20 AM', 2, '15', ' °C']

15  °C
[2014, 11, 2, '12:50 AM', 3, '15', ' °C']

15  °C
[2014, 11, 2, '1:20 A

결손 데이터 칸의 정상 시각 >> 11:50 PM
사이트에서 스크래핑한 시각 >> 12:20 AM
[2014, 11, 5, '11:50 PM', 1, '8 ', '°C']

8  °C
[2014, 11, 5, '12:20 AM', 2, '8 ', '°C']

9  °C
[2014, 11, 5, '12:50 AM', 3, '9 ', '°C']

8  °C
[2014, 11, 5, '1:20 AM', 4, '8 ', '°C']

8  °C
[2014, 11, 5, '1:50 AM', 5, '8 ', '°C']

7  °C
[2014, 11, 5, '2:20 AM', 6, '7 ', '°C']

7  °C
[2014, 11, 5, '2:50 AM', 7, '7 ', '°C']

7  °C
[2014, 11, 5, '3:20 AM', 8, '7 ', '°C']

7  °C
[2014, 11, 5, '3:50 AM', 9, '7 ', '°C']

7  °C
[2014, 11, 5, '4:20 AM', 10, '7 ', '°C']

7  °C
[2014, 11, 5, '4:50 AM', 11, '7 ', '°C']

7  °C
[2014, 11, 5, '5:20 AM', 12, '7 ', '°C']

7  °C
[2014, 11, 5, '5:50 AM', 13, '7 ', '°C']

7  °C
[2014, 11, 5, '6:20 AM', 14, '7 ', '°C']

7  °C
[2014, 11, 5, '6:50 AM', 15, '7 ', '°C']

7  °C
[2014, 11, 5, '7:20 AM', 16, '7 ', '°C']

7  °C
[2014, 11, 5, '7:50 AM', 17, '7 ', '°C']

7  °C
[2014, 11, 5, '8:20 AM', 18, '7 ', '°C']

8  °C
[2014, 11, 5, '8:50 AM', 19, '8 ', '°C']

8  °C
[2014, 11, 5, '9:20 AM', 20, '8 ', '°C'

10  °C
[2014, 11, 8, '6:50 AM', 15, '10', ' °C']

결손 데이터 칸의 정상 시각 >> 7:20 AM
사이트에서 스크래핑한 시각 >> 7:50 AM
[2014, 11, 8, '7:20 AM', 16, '10', ' °C']

10  °C
[2014, 11, 8, '7:50 AM', 17, '10', ' °C']

11  °C
[2014, 11, 8, '8:20 AM', 18, '11', ' °C']

12  °C
[2014, 11, 8, '8:50 AM', 19, '12', ' °C']

결손 데이터 칸의 정상 시각 >> 9:20 AM
사이트에서 스크래핑한 시각 >> 9:50 AM
[2014, 11, 8, '9:20 AM', 20, '12', ' °C']

13  °C
[2014, 11, 8, '9:50 AM', 21, '13', ' °C']

13  °C
[2014, 11, 8, '10:20 AM', 22, '13', ' °C']

결손 데이터 칸의 정상 시각 >> 10:50 AM
사이트에서 스크래핑한 시각 >> 11:50 AM
[2014, 11, 8, '10:50 AM', 23, '13', ' °C']

결손 데이터 칸의 정상 시각 >> 11:20 AM
사이트에서 스크래핑한 시각 >> 11:50 AM
[2014, 11, 8, '11:20 AM', 24, '13', ' °C']

14  °C
[2014, 11, 8, '11:50 AM', 25, '14', ' °C']

14  °C
[2014, 11, 8, '12:20 PM', 26, '14', ' °C']

14  °C
[2014, 11, 8, '12:50 PM', 27, '14', ' °C']

14  °C
[2014, 11, 8, '1:20 PM', 28, '14', ' °C']

14  °C
[2014, 11, 8, '1:50 PM', 29, '14', ' °C']

14  °C
[2014, 11, 8, '2:20 PM', 30, '14', ' °C']

14  °C

AttributeError: 'NoneType' object has no attribute 'text'

#### 1년 기준 12달치 통합 저장 코드

In [ ]:
# 보류